### Imports and Constants

In [11]:
# Uncomment on first ever run to install dependencies.

# If static analysis does not recognize any of your imports:
# Uncomment the following line, then press ctrl-shift-p and enter "Developer:Reload Window" 

%pip install opencv-python numpy tensorflow scikit-learn face-recognition pillow xlsxwriter dlib

Note: you may need to restart the kernel to use updated packages.


In [12]:

import cv2
import tensorflow as tf
import numpy as np

import os
import pickle,struct

import face_recognition
import tkinter as tk
import tkinter.messagebox
from PIL import Image,ImageTk 
import sqlite3

import constants,signals

import threading
from threading import Thread,current_thread ,RLock

import pandas as pd


import logging, sys
from io import BytesIO 

from datetime import datetime,date
import datetime
from time import sleep
# from pathlib import Path
from my_net_utils import *
from socket import *
import tkinter.filedialog 
import re as regex



if 'Environment_Server' not in os.getcwd():
    os.chdir('Environment_Server')
    

###Constants 

In [13]:
#TODO: Unify these in one pickle for coherency, preferably as a dataframe or something

LABELS_TO_FACES_DICT_PICKLE_FILENAME = 'labels_to_face_encodings_dict.pkl'
LAST_RUN_PICKLE_FILENAME = 'last_run_exit_time.pkl'
ACCURACY_STATS_PICKLE_FILENAME = 'accuracy_stats.pkl'

STUDENTS_TABLE_NAME = 'students'
SCHEDULE_TABLE_NAME = 'schedule'
ATTENDANCE_TABLE_NAME = 'attendance'
SUBJECTS_TABLE_NAME = 'subjects'
MAX_CONN = 5


###Error Logging

In [14]:
LOGFILE_NAME = 'ServerErrorLog.log'

logging.basicConfig(stream=sys.stderr,encoding='utf-8',level=logging.DEBUG,format='%(asctime)s %(levelname)s %(name)s %(message)s')

#logging.basicConfig(stream=sys.stderr,encoding='utf-8',level=logging.DEBUG)
logger = logging.getLogger(__name__)
handler = logging.FileHandler(LOGFILE_NAME, 'a', 'utf-8')
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(name)s %(message)s'))
logger.addHandler(handler)
logging.getLogger('PIL').setLevel(logging.WARNING) #Suppress pointless Pillow warnings

def warn_user_and_log_event(error : Exception, 
                            * ,
                 level:int = logging.ERROR, heading="Error" , 
                 logfile_msg : str = None, user_msg : str = None,
                 suppress : bool = False, suppress_condition = '' ) ->None:
    """
    Shows user a message about a warning (unless suppress is set to True), 
    afterwards, write  to a log file with a timestamp for later inspection

    Args:
        error (Exception): Exception raised, if any
        level (str): Which level from a predefined set of levels (in logging) is to be used
        logfile_msg (str): Message to add to log. If none set, use exception text instead.
        user_msg (str): Message to show to user. If none set, use str(error) instead.
        log_event (bool): Choose whether or not to log. Log by default
        suppress (bool): Choose whether or not to show error to user. False by default.
        suppress_condition (str): Suppress only if the substring specifies appears in the exception raised. If not set, suppress unconditionally.
    """

    log_message = logfile_msg if logfile_msg else error
    
    if not error and not user_msg:
        user_msg = logfile_msg    

    level_is_valid = level in range(logging.DEBUG,logging.CRITICAL + 1, 10) 
    
    if not level_is_valid:
        tkinter.messagebox.showerror(title = "Invalid logger call" , message=f'You tried logging an invalid level!')
        level = logging.ERROR #keep a default value  
    
    should_show_info = level >= logging.WARNING

    logger.log(level, msg = log_message , exc_info = should_show_info)
    print('\n\n',file=handler.stream)

    display_message = user_msg if user_msg else str(error)

    if not suppress or suppress_condition not in str(error):
        if level >= logging.WARNING:
            tkinter.messagebox.showerror(title = heading , message=f'An error has occurred: {display_message}' )
        elif level == logging.WARNING:
            tkinter.messagebox.showwarning(title = heading , message=f'Warning: {display_message}' )
        else:
            tkinter.messagebox.showinfo(title = heading , message=f'Alert: {display_message}' )


    return


### Existing encodings/Mappings Retrieval

In [15]:
def load_face_encodings_from_photos(directory='Known_Faces') -> dict:
    if os.path.isfile(LABELS_TO_FACES_DICT_PICKLE_FILENAME):
        try:
            labels_to_encodings = pickle.load(open(LABELS_TO_FACES_DICT_PICKLE_FILENAME,'rb'))
        except Exception as e:
            warn_user_and_log_event(e,level=logging.INFO,logfile_msg="No encodings detected. Rebuilding.",suppress=True)
            labels_to_encodings = {}
    else:
        labels_to_encodings = {}

    for file in os.listdir(directory):
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'): 
                file_name, _file_ext = os.path.splitext(file)
                stu_id , stu_name  = file_name .split('_')
                stu_name = regex.sub(r'-',' ',stu_name)
                
                assert stu_id.isnumeric()
                label = stu_id,stu_name

                if label not in labels_to_encodings.keys() : #Do not overwrite existing encodings
                    image_path = os.path.join(directory, file)
                    # Load image and generate face encoding 
                    image = face_recognition.load_image_file(image_path)
                    encoding = face_recognition.face_encodings(image)
                    
                    #if face has been identified. 
                    if len(encoding) > 0:
                            labels_to_encodings[label] = encoding[0]
                                    
                                    
    pickle.dump(labels_to_encodings,open(LABELS_TO_FACES_DICT_PICKLE_FILENAME,'wb'))

    return labels_to_encodings




### Wipe all existing data

In [21]:

# if 'Environment_Server' not in os.getcwd():
#     os.chdir('Environment_Server')
    
# %run wipe.py --dbpath SQL_db/class.db --pickle $LABELS_TO_FACES_DICT_PICKLE_FILENAME
# if os.path.exists('students_info.xlsx'): 
#     os.chmod('students_info.xlsx',0o666)
#     os.remove('students_info.xlsx')

['students', 'schedule', 'subjects', 'attendance']


### MAIN LOOP

In [17]:
class ServerApp:
    
    last_exit_time = pickle.load(open(LAST_RUN_PICKLE_FILENAME,'rb')) if os.path.isfile(LAST_RUN_PICKLE_FILENAME) else datetime.now()
    time_of_launch = datetime.now()


    if os.path.isfile(ACCURACY_STATS_PICKLE_FILENAME):
        __correct_predictions, __total_predictions = pickle.load(open(ACCURACY_STATS_PICKLE_FILENAME,'rb'))
    else:
        __correct_predictions , __total_predictions = 0 , 0

    __accuracy = __correct_predictions/__total_predictions if __total_predictions > 0 else 0 
   
    def ungraceful_exit(self,exc : Exception):
    #TODO: Feel unnecessary. If I don't have any good reason to keep this I'm removing!   
        if self.db_connection:
            self.db_connection.close()
        if self.server_sock:
            self.server_sock.close()
        if self.root:
            self.root.destroy()
        raise exc
        
    def __init__(self) -> None:
        self.initialize_values()
        self.make_root()

    def make_root(self):
        self.root = tk.Tk()
        self.root.title('Start Page')
        self.root.protocol('WM_DELETE_WINDOW',self.quit_app)

        self.root_widgets : list[tk.Widget] = []

        self.root_widgets.extend(
            [
                # tk.Button(self.root, text="Import Excel file with all sheets (Not implemented)", command = None),
                tk.Button(self.root,text='Import Subject Names',command = lambda: self.populate_table_from_spreadsheet(SUBJECTS_TABLE_NAME)),
                tk.Button(self.root,text='Import Schedule Sheet',command = lambda: self.populate_table_from_spreadsheet(SCHEDULE_TABLE_NAME)),
                tk.Button(self.root,text='Import Registration Sheet',command = lambda: self.populate_table_from_spreadsheet(ATTENDANCE_TABLE_NAME)),
                tk.Button(self.root, text="Start Attendance", command=self.start_attendance),
                tk.Button(self.root, text="Force write Absences",command = lambda : self.write_absences(True) ),
                tk.Button(self.root, text="Exit", command = self.quit_app)
             ]
        ) 

    def initialize_values(self):       
        self.server_sock = socket()
        self.server_sock.setblocking(False)
        self.server_sock.setsockopt(SOL_SOCKET,SO_KEEPALIVE,1)
        self.server_sock.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
        self.absences_were_written = False
        self.day_today = datetime.today().strftime('%A')[0:3].join('%%')
        
        self.number_connections_served = 0
        self.connections = [] 
        self.connection_handler_threads : list[Thread] = []
        self.lock = RLock()
        #An Rlock is required here since accept_connections spawns 
        #handle_client threads, both of which use the same lock.
        self.keep_threads_running = threading.Event() 
        #TODO: Think about making it do the opposite (AKA call it self.stop_threads) and invert the logic checks
        # this is because we might need the .wait() functionality that way. 
        self.keep_threads_running.clear()
        self.error_in_thread = None

        self.path_db = 'SQL_db/class.db'
        
    def launch(self):
        self.server_sock.bind(('',PORT))
        self.db_connection = sqlite3.connect(self.path_db ,
                                             detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES,
                                             check_same_thread=False)
        self.db_cursor = self.db_connection.cursor()
        self.db_cursor.execute('PRAGMA foreign_keys = ON') #FKs not on by default in sqlite
        print('DB connection created')
        
        try:
            self.create_tables()
            self.load_model()



        except sqlite3.IntegrityError as integrity_violation:
            warn_user_and_log_event(integrity_violation,level=logging.CRITICAL)
            self.ungraceful_exit(integrity_violation)

        except FileNotFoundError as no_info:
            warn_user_and_log_event(no_info,
                                    level=logging.INFO,
                                    suppress=True)
            

        for widget in self.root_widgets:
            widget.pack()

        self.root.mainloop()

    def populate_table_from_spreadsheet(self,table_name):

        if table_name == SCHEDULE_TABLE_NAME and self.table_isempty(SUBJECTS_TABLE_NAME):
            tkinter.messagebox.showinfo('Must import subjects first',
                                        'Please import the list of subjects before attempting to import schedule.')
            return


        if table_name == ATTENDANCE_TABLE_NAME and self.table_isempty(SCHEDULE_TABLE_NAME):
            tkinter.messagebox.showinfo('Must import schedule first',
                                        'Please import the schedule before attempting to import registration.')
            return

        excel_extensions = ['.xls', '.xlsx', '.xlsm', '.xlsb', '.odf', '.ods', '.odt']
        file_name = tkinter.filedialog.askopenfilename(
            filetypes = [('Excel-readable File', f'*{extension}') for extension in excel_extensions + ['.csv']]  )
        
        if not file_name: 
            return
        
        file_extension = os.path.splitext(file_name)[-1]
        
        is_excel = False

        if file_extension in excel_extensions:
            is_excel = True

        elif file_extension == '.csv':
            is_csv = True

        else:
            #Happens when a file is not filtered out by filetypes, such as internet shortcuts
            tkinter.messagebox.showwarning('Invalid choice',
                                           'File chosen cannot be read for data. Please use a valid excel or .csv file.')
            return
        
        try:
            if is_excel:
                df_table = pd.read_excel(file_name)
            elif is_csv:
                df_table = pd.read_csv(file_name)

        except OSError as file_inaccessible:
            warn_user_and_log_event(file_inaccessible,level=logging.WARNING,
                                    user_msg = "The file you selected could not be read. "
                                    "It's possible it's already open or that you do not have "
                                    "the required privileges to access it.",
                                    suppress=False)
            return

        except ValueError as file_was_not_spreadsheet:
            warn_user_and_log_event(file_was_not_spreadsheet
                                    ,level=logging.INFO,
                                    user_msg = "The file you selected was of a valid format, but did not contain readable tables.",
                                    suppress=False)
            return
        

        try:
            if table_name == ATTENDANCE_TABLE_NAME:
                table_headings = df_table.columns

                if ServerApp.__BOOL_ABSENT not in table_headings:
                    df_table[ServerApp.__BOOL_ABSENT] = 'TRUE'
                
                if ServerApp.__NUM_ABSENT not in table_headings:
                    df_table[ServerApp.__NUM_ABSENT] = 0
                    

            #There's a way to do this better by combining two dfs and removing duplicates, but let's not worry about that yet
            df_table.to_sql(name=table_name,con=self.db_connection,if_exists='append',index=False) 

        except sqlite3.IntegrityError as pk_collision_or_fk_mismatch:
            warn_user_and_log_event(error = pk_collision_or_fk_mismatch,
                                    user_msg = f"Error in table {table_name}: {pk_collision_or_fk_mismatch}",
                                    level=logging.WARNING)
        
        except sqlite3.OperationalError as shape_mismatch:
            warn_user_and_log_event(shape_mismatch,
                                    user_msg=f'Invalid format for table {table_name}',
                                    level=logging.INFO,
                                    suppress=False)

        except Exception as misc_exc:
            warn_user_and_log_event(misc_exc,
                                    level=logging.ERROR,
                                    suppress=False)
            return  
    
    def write_absences(self,frombutton=False):
        
        # Guard clauses

        if self.absences_were_written:
            tkinter.messagebox.showwarning(title="Already wrote absneces!",
                                           message='Absences were already written in this session.')
            return
        
        if frombutton:
            if not tkinter.messagebox.askyesno(title='Really dump absences?',
                                               message='It appears you are trying to manually write absences.'
                                               ' Only do this if you are very sure that you really need to.'):
                return
        
        hours_since_last_exit = (ServerApp.last_exit_time - datetime.now()).seconds / (60**2)
        
        if not hours_since_last_exit > 18: #arbitrarily chosen "reasonable" time delta
            return 
        
        (is_end_of_day,) = self.db_cursor.execute(f"SELECT max({ServerApp.__TIME_END}) > time('now','localtime') FROM {SCHEDULE_TABLE_NAME}").fetchone()

        if not is_end_of_day:
            return
        
        # End of guard clauses


        pickle.dump(datetime.now() , open(LAST_RUN_PICKLE_FILENAME,'wb'))
        
        df_attendance = pd.read_sql(f'SELECT * FROM {ATTENDANCE_TABLE_NAME}',self.db_connection,index_col=[ServerApp.__STU_ID,ServerApp.__SBJ_ID])

        attendance_filename = f'attendance_{datetime.now().strftime('%Y-%m-%d')}.xlsx'
        if os.path.exists(attendance_filename):
            return
        
        with pd.ExcelWriter(attendance_filename,engine='xlsxwriter',) as writer:
            df_attendance.to_excel(writer,sheet_name=f'{ATTENDANCE_TABLE_NAME}',merge_cells=False)

        self.db_cursor.execute(f'''SELECT {ServerApp.__SBJ_ID} , {ServerApp.__SEC} 
                                    FROM {SCHEDULE_TABLE_NAME}
                                    WHERE {ServerApp.__DAYS} LIKE ?''',(self.day_today.join('%%'),))
        
        todays_lectures = self.db_cursor.fetchall()

        if todays_lectures:
            for column in todays_lectures:
                self.db_cursor.execute(f'''UPDATE {ATTENDANCE_TABLE_NAME} 
                                    SET {ServerApp.__NUM_ABSENT} = {ServerApp.__NUM_ABSENT} + 1 
                                    WHERE {ServerApp.__BOOL_ABSENT} = TRUE
                                    AND {ServerApp.__SBJ_ID} = ?
                                    AND {ServerApp.__SEC} = ?''',
                                    column)
        
        self.db_cursor.execute(f"UPDATE {ATTENDANCE_TABLE_NAME} set {ServerApp.__BOOL_ABSENT} = TRUE") #Reset all absences
        
        self.db_connection.commit()

        self.absences_were_written = True


    def quit_app(self):
        pickle.dump((ServerApp.__correct_predictions, ServerApp.__total_predictions) , open(ACCURACY_STATS_PICKLE_FILENAME,'wb'))

        hours_this_session = (datetime.now() - ServerApp.time_of_launch).seconds / (60**2)
        
        # Allow for maintenance runs up to 2hr. 
        # Circumvent the time limit by just restarting the program before 2 hours pass
        # if maintenance take longer than 2 hours
        # It's assumed the program will never be need to run less than 2 hours for registration purposes.
        # But this value can be adjusted later 
        MINIMUM_DAY_LENGTH_IN_HOURS = 2
        
        if hours_this_session > MINIMUM_DAY_LENGTH_IN_HOURS:
            self.write_absences()

        ServerApp.__accuracy = ServerApp.__correct_predictions/ServerApp.__total_predictions if ServerApp.__total_predictions > 0 else 0 
        
        with open(LOGFILE_NAME,'a') as f:
            f.write(f'\nCumulative prediction accuracy = {ServerApp.__accuracy : .0%}\n')

        #Disallow writing by user     
        # os.chmod(attendance_filename ,0o444) 
        #SCRAPPED: We want the ability to let users edit.
        
        self.db_connection.close()
        self.server_sock.close()
        self.root.quit()
        self.root.destroy()

    
    __STU_ID = "student_id"
    __STU_NAME = "student_name"
    __SBJ_ID = "subject_id"
    __SEC = "section"
    __ROOM = "room"
    __DAYS = "days"
    __TIME_ST = "start_time"
    __TIME_END = "end_time"
    __BOOL_ABSENT = "is_absent"
    __NUM_ABSENT = "num_absences"
    __SBJ_NAME = "subject_name"

    def create_tables(self): #always ensure tables are present 

        execute_statements = [f'''CREATE TABLE IF NOT EXISTS "{STUDENTS_TABLE_NAME}"(
            "{ServerApp.__STU_ID}"	INTEGER,
            "{ServerApp.__STU_NAME}"	TEXT COLLATE NOCASE,
            PRIMARY KEY("{ServerApp.__STU_ID}") ON CONFLICT IGNORE 
            )
        ''',

        #Assumption: No overlaps will occur, and no scheduling errors in general will happen 
        f'''CREATE TABLE IF NOT EXISTS "{SCHEDULE_TABLE_NAME}"(
            "{ServerApp.__SBJ_ID}"    INTEGER,
            "{ServerApp.__SEC}"	INTEGER,
            "{ServerApp.__ROOM}"	TEXT NOT NULL,
            "{ServerApp.__DAYS}"	TEXT NOT NULL COLLATE NOCASE,
            "{ServerApp.__TIME_ST}"	DATETIME NOT NULL,
            "{ServerApp.__TIME_END}"	DATETIME NOT NULL,
            
            UNIQUE("{ServerApp.__ROOM}","{ServerApp.__DAYS}","{ServerApp.__TIME_ST}") ON CONFLICT FAIL,
            PRIMARY KEY("{ServerApp.__SBJ_ID}","{ServerApp.__SEC}") ON CONFLICT REPLACE 
            FOREIGN KEY("{ServerApp.__SBJ_ID}") REFERENCES {SUBJECTS_TABLE_NAME}("{ServerApp.__SBJ_ID}")
            )
        ''',
        
         f'''CREATE TABLE IF NOT EXISTS "{ATTENDANCE_TABLE_NAME}"(
            "{ServerApp.__STU_ID}"	INTEGER,
            "{ServerApp.__SBJ_ID}"	INTEGER,
            "{ServerApp.__SEC}"	INTEGER,
            "{ServerApp.__BOOL_ABSENT}" BOOLEAN DEFAULT TRUE,
            "{ServerApp.__NUM_ABSENT}" INTEGER DEFAULT 0,
            
            PRIMARY KEY("{ServerApp.__STU_ID}","{ServerApp.__SBJ_ID}") ON CONFLICT FAIL,

            FOREIGN KEY("{ServerApp.__STU_ID}") REFERENCES "{STUDENTS_TABLE_NAME}"("{ServerApp.__STU_ID}"),
            FOREIGN KEY("{ServerApp.__SBJ_ID}","{ServerApp.__SEC}") REFERENCES "{SCHEDULE_TABLE_NAME}"("{ServerApp.__SBJ_ID}","{ServerApp.__SEC}") )
            ''',
        f'''CREATE TABLE IF NOT EXISTS {SUBJECTS_TABLE_NAME}(
            "{ServerApp.__SBJ_ID}" INTEGER,
            "{ServerApp.__SBJ_NAME}" TEXT UNIQUE NOT NULL COLLATE NOCASE,
            
            PRIMARY KEY("{ServerApp.__SBJ_ID}") ON CONFLICT FAIL)
        '''
        ]

        self.db_cursor.executescript('; '.join(execute_statements))
        self.db_connection.commit()  

    def FUTURE_get_student_absences(self,student_id):
        try:
            absences = self.db_cursor.execute(f'SELECT ({ServerApp.__STU_ID}, {ServerApp.__SBJ_ID} , {ServerApp.__NUM_ABSENT})\
                                              FROM {ATTENDANCE_TABLE_NAME} \
                                              WHERE "{ServerApp.__STU_ID}" = ?',
                                              (student_id,)).fetchall()
            return [record for record in absences] 
        
        except Exception as misc_exc:
            warn_user_and_log_event(misc_exc,level=logging.WARNING)


    def accept_connections(self):
        while self.keep_threads_running.is_set(): 
            try:
                with self.lock:
                    num_active_conns = len(self.connections)

                if num_active_conns < MAX_CONN:
                    client_connection, raddr = self.server_sock.accept() 
                    client_connection.settimeout(SV_TIMEOUT)
                    with self.lock:
                        self.connections.append(client_connection)
                        self.number_connections_served += 1
                        r_ip, _r_port = raddr
                        r_ip = r_ip.split('.')
                        building = r_ip[-2]
                        match building:
                            case '100':
                                building_code = 'EE'
                            case '150':
                                building_code = 'BU'
                            case '200':
                                building_code = 'CS'
                            case _:
                                building_code = ''

                        #Allow to work in localhost
                        if r_ip[0] == '127':
                            offset = 105
                            building_code = 'EE'
                        else:
                            offset = 100
                        
                        room_name = building_code + str( offset + int(r_ip[-1]) ) 

                        t = Thread(
                        target=self.handle_client,
                        args=(client_connection,
                              raddr,
                              room_name),
                        name=f'Room_{room_name}',
                        daemon=False)
                        self.connection_handler_threads.append(t)
                        self.connection_handler_threads[-1].start()
                else:
                    sleep(1)
                    #TODO: This might be better done with some combination of threading.Event and wait() 
                    # but for now let's keep it since it works
                    
            except BlockingIOError as e:
                sleep(1.5)
                pass # Suppress error

        for thread in self.connection_handler_threads:
            print(f'joining thread {thread.name}')
            thread.join(timeout = 30)
            if not thread.is_alive():
                print(f'joined thread {thread}')
            else:
                with self.lock:
                    i = self.connection_handler_threads.index(thread)
                    self.connections[i].shutdown(SHUT_RDWR)
                    self.connections[i].close()
        
    
    def handle_client(self, connection : socket, remote_address : str, room_name : str):
        
        client_quit = False
        
        CRITERIA = f'''time('now','localtime') BETWEEN 
        strftime('%H:%M',{ServerApp.__TIME_ST}) and strftime('%H:%M', {ServerApp.__TIME_END},'-15 minutes' )
        AND  
        {ServerApp.__DAYS} LIKE ?
        AND 
        {ServerApp.__ROOM} = ?'''
        CRITERIA_INSERTS = (self.day_today,room_name)
        # Room + Time + Day combination is unique so this only ever returns one tuple
            # This scheme assumes no scheduling conflicts happen (handled earlier in pipeline)

        already_checked = []

        try:                          
            if room_name.isnumeric():
                #Reject connections from unknown IPs
                connection.shutdown(SHUT_RDWR)
                connection.close()
                return
            
            remote_ip,_= remote_address

                        
            self.set_status_threadsafe(f'Accepted connection from {remote_ip} in room {room_name}')
            with open(LOGFILE_NAME,'at') as f:
                 f.write(f'\n{room_name} connected at {datetime.now().isoformat()}\n')
            
            while self.keep_threads_running.is_set():
                error_in_thread = None
                try:

                    #TODO: optimize this a little. A SQL trigger would be good.

                            
                    if self.keep_threads_running.is_set():
                        try: 
                            msg_in = connection.recv(4) 
                            msg_in_str = msg_in.decode('utf-8') 
                            if msg_in_str == signals.QUIT or not self.keep_threads_running.is_set():
                                self.set_status_threadsafe(f'Connection {remote_address} has quit normally.')
                                connection.shutdown(SHUT_RDWR)
                                connection.close()
                                client_quit = True
                                break
                            
                        #FIXME: This is common enough that it would fare better as an if statement, not an exception
                        except ValueError as not_quit:
                            pass
                        except Exception as misc_exc:
                            self.set_status_threadsafe(f'Error in thread {current_thread().name}: {misc_exc}')
                            warn_user_and_log_event(misc_exc,suppress=True)
                            error_in_thread = misc_exc
                            break

                    with self.lock:
                        ongoing_lecture = self.db_cursor.execute(f'''SELECT {ServerApp.__SBJ_ID},{ServerApp.__SEC}
                                                    FROM {SCHEDULE_TABLE_NAME}
                                                    WHERE time('now','localtime') BETWEEN 
                                                    strftime('%H:%M',{ServerApp.__TIME_ST},'-2 minutes')
                                                    AND
                                                    strftime('%H:%M', {ServerApp.__TIME_END},'-15 minutes' )
                                                    AND  
                                                    {ServerApp.__DAYS} LIKE ?
                                                    AND 
                                                    {ServerApp.__ROOM} = ?''',
                                                    (self.day_today,room_name)).fetchone() 

                    # print(ongoing_lecture,self.day_today,room_name,sep='\n')
                    if not ongoing_lecture:
                        already_checked = []

                    if self.return_button.winfo_exists() and self.return_button.winfo_viewable():
                        with self.lock:
                            self.return_button.pack_forget()
                            #We are enforcing this with an iron grip I am afraid. 

                    
                    (image_size,) = struct.unpack('!I',msg_in)
                    if self.keep_threads_running.is_set():
                        padded_send(connection,signals.SIZE_RECEIVED)
                    if self.keep_threads_running.is_set():
                        image_raw = receive_normal(connection,image_size)
                    
                    png_bytes = BytesIO()
                    png_bytes.write(image_raw)

                    start_of_buffer = 0 
                    png_bytes.seek(start_of_buffer)

                    student_image = Image.open(png_bytes)

                    image_for_model = np.array(student_image)

                    image_for_model = cv2.cvtColor(image_for_model,cv2.COLOR_RGB2BGR)

                    photo_img = ImageTk.PhotoImage(image=student_image)
                    
                    with self.lock:
                        self.student_image.image = photo_img #Prevents image being wiped on disconnect  
                        self.student_image.config(image=photo_img)
                    
                    last_predicted_student_id, last_predicted_student_name = self.predict_face(image_for_model)
                    last_predicted_student_id = str(last_predicted_student_id) #ensure it's a string (it should be, but it is fully numeric)
                    
                    with self.lock:
                        self.update_textbox('\n'.join((last_predicted_student_name,last_predicted_student_id)) )  
                        ServerApp.__total_predictions += 1


                    if last_predicted_student_id:
                        padded_send(connection,f'{last_predicted_student_id} -- {last_predicted_student_name}')

                    else:
                        # Assuming no face being detected happens only due to operator error
                        if last_predicted_student_name == constants.NONE: 
                            ServerApp.__total_predictions -= 1
                        
                        padded_send(connection,last_predicted_student_name) #Either UNKNOWN and NONE 
                        continue
                    
                    #TODO: There has got to be a better way
                    if not self.keep_threads_running.is_set():
                        break
                    
                    client_response = receive_strip(connection) #Client responds if name was correct or not
                    
                    last_prediction_correct = (client_response == signals.YES) 

                    if not last_prediction_correct:
                        continue

                    if last_predicted_student_name: #do not append none or unknown to already_checked!
                        
                        if last_predicted_student_id in already_checked:
                            padded_send(connection,signals.ALREADY_SEEN(last_predicted_student_id))
                            with self.lock: ServerApp.__correct_predictions += 1
                            continue

                        already_checked.append(last_predicted_student_id)
                        

                    
                    with self.lock:
                        ServerApp.__correct_predictions += 1
                        try:    
                            error_in_writing_db = None                            
                            
                            if not ongoing_lecture:
                                log_result = signals.NO_CLASS_IN_SESSION
                                
                            else:
                                sbj_id_ongoing, section_ongoing = ongoing_lecture

                                (sbj_name_ongoing,) = self.db_cursor.execute(f''' SELECT {ServerApp.__SBJ_NAME} 
                                                                  FROM {SUBJECTS_TABLE_NAME} 
                                                                  WHERE {ServerApp.__SBJ_ID} = ?''',
                                                                  (sbj_id_ongoing,)).fetchone()

                                self.db_cursor.execute(f'''UPDATE {ATTENDANCE_TABLE_NAME} 
                                                        SET {ServerApp.__BOOL_ABSENT} = ? 
                                                        WHERE {ServerApp.__STU_ID} = ?
                                                        AND {ServerApp.__SBJ_ID} = ?
                                                        AND {ServerApp.__SEC} = ? ''',
                                                            ('FALSE',
                                                            last_predicted_student_id,
                                                            sbj_id_ongoing,
                                                            section_ongoing) )
                                
                                assert self.db_cursor.rowcount < 2
                                #FIXME: Remove in release. This is entirely for debugging

                                if self.db_cursor.rowcount > 0:
                                    log_result = signals.SUCCESSFULLY_MARKED_PRESENT(last_predicted_student_id,sbj_name_ongoing,section_ongoing)
                                else:
                                    log_result = signals.NOT_ENROLLED(last_predicted_student_id,sbj_name_ongoing,section_ongoing)
                                
                            if self.keep_threads_running.is_set():
                                padded_send(connection,log_result)

                        except Exception as db_exc:
                            error_in_writing_db = db_exc
                            self.db_connection.rollback()
                            
                        else:
                            self.db_connection.commit()

                        finally:
                            if error_in_writing_db:
                                level = logging.ERROR if isinstance(error_in_writing_db,sqlite3.IntegrityError) else logging.CRITICAL
                                warn_user_and_log_event(error_in_writing_db,
                                                        level=level)
                                self.set_status_threadsafe(f'Error in writing to database: {error_in_writing_db}')
                                error_in_thread = error_in_writing_db
                                #If an error happens here there's a programming issue.
           
                except TimeoutError as timeout_error:
                    self.set_status_threadsafe(f'Connection with {remote_ip} timed out')
                    warn_user_and_log_event(timeout_error,level=logging.DEBUG,suppress=True)
                    error_in_thread = timeout_error
                    client_quit = True
                    
                except ConnectionError as dropped_connection:
                    self.set_status_threadsafe(f'Remote host {remote_ip} unexpectedly disconnected')
                    warn_user_and_log_event(dropped_connection,level=logging.INFO,suppress=True)
                    error_in_thread = dropped_connection
                    client_quit = True

                except Exception as misc_exc:
                    self.set_status_threadsafe(f'Exception in thread {current_thread().name} with connection {remote_ip}\n: {misc_exc}')
                    warn_user_and_log_event(misc_exc,level=logging.CRITICAL,suppress=True)
                    error_in_thread = misc_exc
                
                finally:
                    with self.lock:
                        if self.return_button.winfo_exists() and not self.return_button.winfo_viewable(): #prevent double-packs in case of multiple accesses
                            self.return_button.pack()
                    if error_in_thread and not isinstance(error_in_thread,(ConnectionError,TimeoutError)):
                        self.keep_threads_running.clear()
                        return

        except ConnectionError as err_connection :
            
            self.set_status_threadsafe(err_connection)
            warn_user_and_log_event(err_connection,level=logging.INFO,suppress=True,suppress_condition='not a socket')
        finally:
            try:
                with open(LOGFILE_NAME,'at') as f:
                    f.write(f'\n{room_name} disconnected at {datetime.now().isoformat()}\n')
                if not client_quit and self.keep_threads_running.is_set():
                    connection.shutdown(SHUT_RDWR)
                    connection.close()


            except (TimeoutError,ConnectionError,OSError) as conn_err:
                warn_user_and_log_event(conn_err,log_event=False,level=logging.WARNING,suppress=True)
            except Exception as misc_exc:
                warn_user_and_log_event(misc_exc,level=logging.ERROR,suppress=True)

            with self.lock:
                if connection in self.connections:
                    self.connections.remove(connection)
                
                self.connection_handler_threads.remove(current_thread())
                
                if len(self.connections) == 0 and not error_in_thread:
                    self.set_status_threadsafe('No one connected. Feel free to exit!')
            return 

    def set_status_threadsafe(self,message):
        with self.lock:
            self.status.set(str(message))   
            
    def table_isempty(self,table_name):
        self.db_cursor.execute(f'SELECT * FROM {table_name}')
        data = self.db_cursor.fetchone()
        return False if data is not None else True
    #table is not empty if non-null data exists
    
    def start_attendance(self):
        #Guard clause to check if system will work. Do not start if not.
        for table in (SCHEDULE_TABLE_NAME,ATTENDANCE_TABLE_NAME):
            if self.table_isempty(table):
                tkinter.messagebox.showerror(title='Table Empty' , message=f'Table {table} is empty. Attendance system will not work.')
                return

        self.attendance_window = tk.Toplevel(self.root)        
        self.root.withdraw()
        
        #Disable the exit button. I want to have full control over when to show it.
        self.attendance_window.protocol('WM_DELETE_WINDOW',lambda : None )
        self.server_sock.listen(MAX_CONN) 

        self.student_image = tk.Label(master=self.attendance_window)
        self.student_image.pack()

        photoimg= ImageTk.PhotoImage(image=Image.open('Placeholder.png'))  # Convert PIL Image to ImageTk 
        self.student_image.config(image=photoimg)
        self.student_image.image = photoimg 

        self.status = tk.StringVar(self.attendance_window,value=f'Listening on {self.server_sock.getsockname()}')
        self.status_label = tk.Label(self.attendance_window,textvariable=self.status)
        self.status_label.pack()

        self.last_student_name_label = tk.Label(self.attendance_window,text="Last detected student:")
        self.last_student_name_label.pack()
        
        self.last_student_textbox = tk.Text(self.attendance_window,height=2,width=50)
        self.last_student_textbox.config(state=tk.DISABLED)
        self.last_student_textbox.pack()

        self.return_button = tk.Button(self.attendance_window,command=self.return_to_main,text='Return to main menu')
        self.return_button.pack()

        self.attendance_window.lift()
        self.keep_threads_running.set()

        self.accepter_thread = Thread(target=self.accept_connections,name="Thread_Client",daemon=False)
        
        self.accepter_thread.start()
            
    def return_to_main(self): 
        
        with self.lock:
            active_connections = len(self.connections)

        if active_connections > 0:
            self.set_status_threadsafe("Can't go back when people are connected!")
            return
        
        
        self.keep_threads_running.clear()
        self.accepter_thread.join(timeout= 1.25 * SV_TIMEOUT)
        self.attendance_window.destroy()
        self.root.deiconify()

    def update_textbox(self,new_text : str,textbox : tk.Text = None) -> None:
        if textbox == None:
            textbox = self.last_student_textbox
        textbox.config(state=tk.NORMAL)
        textbox.delete(constants.TK_START_INDEX,tk.END)
        textbox.insert(constants.TK_START_INDEX,new_text)
        textbox.config(state=tk.DISABLED)

    def load_model(self) -> None:
        self.labels_to_encodings = load_face_encodings_from_photos()
        print('Model loaded')
        self.known_face_labels, self.known_face_encodings = list(self.labels_to_encodings.keys()) , list(self.labels_to_encodings.values()) 

        self.db_cursor.executemany(f'INSERT INTO {STUDENTS_TABLE_NAME} VALUES (?,?)',self.known_face_labels)
        self.db_connection.commit()


        df_students = pd.read_sql(f'SELECT * FROM {STUDENTS_TABLE_NAME}',self.db_connection,index_col=ServerApp.__STU_ID)
        
        if os.path.exists('students_info.xlsx'): os.chmod('students_info.xlsx',0o666)
        with pd.ExcelWriter('students_info.xlsx',engine='xlsxwriter') as writer:
            df_students.to_excel(writer,sheet_name=STUDENTS_TABLE_NAME,merge_cells=False)

        os.chmod('students_info.xlsx',0o444)

        


    def predict_face(self,image) -> tuple[str,str]:   
        
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations) #this could probably be compressed into one line
        name = constants.NONE
        id_ = ''

        if face_locations:
            name = constants.UNKNOWN
            
            for encoding in face_encodings:
                # Compare the face encoding with known faces
                matches = face_recognition.compare_faces(self.known_face_encodings, encoding)
                
                if True in matches:
                    
                    first_match_index = matches.index(True)
                    id_ , name = self.known_face_labels[first_match_index]
            
        return id_,name
    
    #TODO: To implement this, you'd need to change the communication protocol on both sides. 
        # Still leaving this here in case we ever decide to come back to it!   
    def FUTURE_predict_all_faces(self,image) -> list[tuple[int,str]]:
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        names = [constants.NONE]
        ids_ = ['']

        if face_locations:
            names = [constants.UNKNOWN]
            for encoding in face_encodings:
                # Compare the face encoding with known faces
                matches = face_recognition.compare_faces(self.known_face_encodings, encoding)
                
                if True in matches:
                    names = []
                    ids_ = []
                    for potential_match, match in enumerate(matches):
                        if match:
                            id_,name = self.known_face_labels[potential_match]
                            ids_.append(id_)
                            names.append(name)

        return ids_,names
    


In [18]:
try:
    instance = ServerApp()
    instance.launch()

except Exception as error_in_running:
    warn_user_and_log_event(error_in_running,suppress=False,level=logging.CRITICAL)


finally:
    del instance


DB connection created
Model loaded
Sent:  b'SIZE_RECEIVED###################################################################################################################'
Receiving...
Sent:  b'NONE############################################################################################################################'
Sent:  b'SIZE_RECEIVED###################################################################################################################'
Receiving...
Sent:  b'20180596 -- Yazan Matarweh######################################################################################################'
Receiving...
APPENDED:YES
RECEIVED YES
Sent:  b'No class is currently ongoing in this room. Please try again later.#############################################################'
Sent:  b'SIZE_RECEIVED###################################################################################################################'
Receiving...
Sent:  b'20180596 -- Yazan Matarweh##################

In [19]:
#!jupyter nbconvert --to script ServerDemo.ipynb

In [20]:
#%load_ext mypy_ipython
#%system net session /delete